In [1]:
import os
import glob
import numpy as np
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
from hsr.config import *
from hsr.loading import winsorize_and_standardize_descriptor, styles
from hsr.regression import cross_sectional_regression_one_day, concat_loadings, _normalize_cap_weights
from hsr.analysis import compute_risk_from_panels_rolling, factor_variance_explained_per_asset,factor_variance_explained_portfolio,calc_realized_vol
print(DEFAULT_PATH)

/Users/janiceyan/work/data/hsr


# calculate loadings

In [6]:
# step 1
descriptor_df = pd.read_parquet(os.path.join(DEFAULT_PATH, "intermediate/descriptor.parquet"))
out_fn = os.path.join(DEFAULT_PATH, "intermediate/cap_weights.parquet")
mcap = descriptor_df[descriptor_df["descriptor"] == "market_cap"].pivot(
    index=date_col, columns=identifier, values="val"
)
mcap.to_parquet(out_fn)

zscore_dfs = []
for descriptor in descriptor_df["descriptor"].unique():
    if descriptor == "market_cap": continue  # market_cap is not a descriptor

    df = descriptor_df[descriptor_df["descriptor"] == descriptor].pivot(
        index=date_col, columns=identifier, values="val"
    )
        
    zscore_df = winsorize_and_standardize_descriptor(df, descriptor, mcap)
    zscore_dfs.append(zscore_df)
zscore_df = pd.concat(zscore_dfs, axis=1)


winsorizing and standardizing beta
winsorizing and standardizing volatility_dsd
winsorizing and standardizing volatility_hilo
winsorizing and standardizing momentum_rs
winsorizing and standardizing size
winsorizing and standardizing nonlinear_size
winsorizing and standardizing trading_activity_annual
winsorizing and standardizing trading_activity_quarter
winsorizing and standardizing trading_activity_5y
winsorizing and standardizing growth_payo
winsorizing and standardizing growth_vcap
winsorizing and standardizing growth_agro
winsorizing and standardizing growth_egro
winsorizing and standardizing growth_dele
winsorizing and standardizing earnings_yield_trailing_12m
winsorizing and standardizing value_bp
winsorizing and standardizing value_sp
winsorizing and standardizing value_cfp
winsorizing and standardizing earnings_variability_vern
winsorizing and standardizing earnings_variability_vsales
winsorizing and standardizing earnings_variability_vcfs
winsorizing and standardizing earning

In [7]:
# step 2
loading_dfs = []
for style in styles:
    cols = [s for s in zscore_df.columns if s.startswith(style)]
    df = zscore_df[cols].mean(axis=1)
    df.name = style
    df = df.reset_index().pivot(index=date_col, columns=identifier, values=style)
    df = winsorize_and_standardize_descriptor(df, style, mcap)
    loading_dfs.append(df)
loading_df = pd.concat(loading_dfs, axis=1)
out_fn = os.path.join(DEFAULT_PATH, "intermediate/loadings.parquet")
loading_df.to_parquet(out_fn)


winsorizing and standardizing beta
winsorizing and standardizing volatility
winsorizing and standardizing momentum
winsorizing and standardizing size
winsorizing and standardizing nonlinear_size
winsorizing and standardizing trading_activity
winsorizing and standardizing growth
winsorizing and standardizing earnings_yield
winsorizing and standardizing value
winsorizing and standardizing earnings_variability
winsorizing and standardizing leverage
winsorizing and standardizing dividend_yield


# Construct factor returns

In [11]:
# load data

simple_ret_df = pd.read_parquet(os.path.join(DEFAULT_PATH, "intermediate/simple_return.parquet"))
industry_df = pd.read_parquet(os.path.join(DEFAULT_PATH, "intermediate/industry_one_hot.parquet"))
loading_df = pd.read_parquet(os.path.join(DEFAULT_PATH, "intermediate/loadings.parquet")).reset_index()
mkt_cap = pd.read_parquet(os.path.join(DEFAULT_PATH, "intermediate/cap_weights.parquet"))

all_dates = np.intersect1d(simple_ret_df.index, loading_df[date_col].unique())
all_tickers = np.intersect1d(simple_ret_df.columns, loading_df[identifier].unique())


In [14]:
# organize data and run by day

def run_one_regression(date, all_tickers,
                       simple_ret_df,
                       industry_df,
                       loading_df,
                       mkt_cap
                       ):

    def _empty(S):
        for col in S.columns:
            if S[col].abs().sum() == 0:
                # print(col)
                return True
        return False    
    r = simple_ret_df.loc[date, all_tickers].dropna()
    tickers_ = r.index
    D_ind = industry_df.loc[tickers_]
    X_style = loading_df[loading_df[date_col] == date].drop(columns=[date_col])
    X_style = X_style.set_index(identifier).loc[tickers_].fillna(0.)
    C_cty = pd.DataFrame(np.ones(len(tickers_)), index=tickers_, columns=["country"])
    mcap = mkt_cap.loc[date, tickers_].fillna(0.)

    if _empty(X_style): 
        # print(f"empty style factor detected, skipping {date}")
        return None, X_style
    # print(f"Processing {date}")

    f_ret, e, _ = cross_sectional_regression_one_day(
        r, X_style, D_ind, C_cty, mcap,
        hsigma=prev_sv.pow(0.5) if 'prev_sv' in locals() else None,
    )
    f_ret.name = date
    e.name = date
    return f_ret, e
    

In [15]:
# run

factor_returns = []
specific_returns = []
for date in all_dates[-252:]:   # run one year
    f_ret, e = run_one_regression(date, all_tickers,
                                  simple_ret_df,
                                  industry_df,
                                  loading_df,
                                  mkt_cap)
    if f_ret is None: continue

    factor_returns.append(f_ret)
    specific_returns.append(e)

factor_return_df = pd.concat(factor_returns, axis=1).T
specific_return_df = pd.concat(specific_returns, axis=1).T

# Compute Factor Covaraince & Specific Risk

In [16]:
# I'm using s&p as regime proxy. you can define your own.
regime_proxy = None
regime_proxy_fn =os.path.join(DEFAULT_PATH, "intermediate/sp.csv") 
if os.path.exists(regime_proxy_fn):
    regime_proxy = pd.read_csv(regime_proxy_fn, header=None)
    regime_proxy = pd.Series(regime_proxy[1].values,
                             index=pd.to_datetime(regime_proxy[0].values),
                             name="regime")

factor_cov, regime_multiplier, specific_var = compute_risk_from_panels_rolling(
        factor_return_df,
        specific_return_df.T,
        regime_proxy=regime_proxy,
        cov_format="long",
    )

# Some analysis measurement

In [17]:
date = "2025-10-17" # enter a date

country_df = pd.DataFrame(1, index=industry_df.index, columns=["country"])
X_s = loading_df[loading_df[date_col] == date].drop(columns=[date_col])
X_s = X_s.set_index(identifier)
        
Sigma_f_t = factor_cov[factor_cov["date"] == date]
Sigma_f_t = Sigma_f_t.pivot(index="row_factor",
                            columns="col_factor",
                            values="value")
spec_var_t = specific_var.T.loc[date]

idx = spec_var_t.index
X_t, all_cols, _, _, _ = concat_loadings(X_s, industry_df, country_df, idx)
X_t = pd.DataFrame(X_t, index=idx, columns=all_cols)

mcap = mkt_cap.loc[date, X_s.index].fillna(0.)
mcap = _normalize_cap_weights(mcap, X_s.index).fillna(0.)
realized_ret = simple_ret_df.loc[: date, X_s.index].fillna(0.)


In [18]:
r2_by_asset = factor_variance_explained_per_asset(X_t, Sigma_f_t, spec_var_t)
r2_by_asset.describe()

count    2563.000000
mean        0.905442
std         0.077255
min         0.283117
25%         0.875676
50%         0.923622
75%         0.958268
max         0.993517
Name: R2_factor_explained, dtype: float64

In [24]:
r2_mcap_port, _, _, mcap_implied_tot = factor_variance_explained_portfolio(mcap, X_t, Sigma_f_t, spec_var_t)
mcap_realized_tot = calc_realized_vol(mcap, realized_ret, window_size=252)

print("cap weighted portfolio R2: %.2f, implied vol: %.4f, realized vol: %.4f" % 
      (r2_mcap_port, np.sqrt(mcap_implied_tot), np.sqrt(mcap_realized_tot)))


cap weighted portfolio R2: 0.93, implied vol: 0.0087, realized vol: 0.0133
